In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPool2D, Input

import numpy as np

import matplotlib.pyplot as plt

import cv2

import os

In [ ]:
class Combine(keras.layers.Layer):
    def __init__(self):
        super(Combine, self).__init__()

    def build(self, input_shape):
        self.alpha = tf.Variable(initial_value=tf.zeros_initializer()(shape=(1,), dtype="float32"), trainable=False)
        
    def get_config(self):
        config = super(Combine, self).get_config().copy()
        return config

    def call(self, inputs):
        return self.alpha*inputs[0] + (1 - self.alpha)*inputs[1]

In [ ]:
class ProgressiveGAN(object):
    def __init__(self, latent_dim=128, initial_image_size=4):
        self.__latent_dim = 128
        self.__initial_image_size=4

        self.__discriminator_input = None
        self.__generator_output = None

        self.__input_combine_layer = None
        self.__output_combine_layer = None

        self.__generator = self.__init_generator()
        self.__discriminator = self.__init_discriminator()
        self.__gan = self.__init_gan()
    
    def fit(self, image_generator, lr=.01):
        pass

    def __init_generator(self):
        pass

    def __init_discriminator(self):
        pass
    
    def __init_gan(self):
        pass